In [2]:
from pulp import *
import pulp
import pandas as pd
import numpy as np
import random

In [3]:
#------------------Import file----------------------

# Read the Excel file into a DataFrame
df = pd.read_excel('file.xlsx')
# results = pd.read_excel('./results.xlsx')
# dimension = pd.read_excel('./dataset/product_hierarchy_planning.xlsx')

# Convert the DataFrame to a NumPy array
data = df.values
# dataResults = results.values
# dataDimensions = dimension.values

products = []

# for i in range(len(dataResults)):
#     predictedProfits = dataResults[i][6]
#     predictedProductQuantities = dataResults[i][5]
#     dimension = dataDimensions[i][]
#     productName = dataResults[i][0]

for row in data:
    predictedProfits = row[0]
    predictedProductQuantities = row[1]
    dimension = row[2]
    productName = row[3]

    product = {'productName': productName,'predictedProfits': predictedProfits,'predictedProductQuantities': predictedProductQuantities,  'dimension': dimension }
    products.append(product)

print(products)

[{'productName': 'p1', 'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5}, {'productName': 'p2', 'predictedProfits': 200, 'predictedProductQuantities': 20, 'dimension': 2}, {'productName': 'p3', 'predictedProfits': 300, 'predictedProductQuantities': 30, 'dimension': 1}, {'productName': 'p4', 'predictedProfits': 10, 'predictedProductQuantities': 1, 'dimension': 20}]


In [7]:
#-------------------------------User Input-------------------------------------

#add user input for the multiple variables

minTotalAmount = int(input("Please insert minimum total amount of products: "))
maxTotalAmount = int(input("Please insert maximum  total amount of products: "))

#create two list with min and max amount per product by user insert

minAmountProduct = int(input("Please insert minimum  amount of products : "))
maxAmountProduct  = int(input("Please insert maximum  amount of products : "))

minProductVariety = int(input("Please insert minimum number of product variety: "))
maxProductVariety  = int(input("Please insert maximum  number of product variety: "))

storeDimension  = int(input("Please insert store dimension: "))

In [ ]:
def calculate_profit(products):
    for product in products:
        profit = product['predictedProfits'] / product['predictedProductQuantities']
        product['productProfit'] = profit
    return products

In [ ]:
productTest = calculate_profit(products)
print(productTest)

In [ ]:
# Creation of the ILP model 
model = pulp.LpProblem("ILP_Problem", pulp.LpMaximize)

In [ ]:
# Define the decision variables
decisionVariables = []
def createDictionaryWithDecisionVariable():
    for product in products:
        decisionVariable = {product['productName']:  pulp.LpVariable(product['productName'], lowBound=0, cat='Integer')}
        decisionVariables.append(decisionVariable)
        
createDictionaryWithDecisionVariable()
print(decisionVariables)

In [ ]:
# function to calculate objective function

def objectiveFunction(products):
    model = 0
    count = 0
    for product in products:
        model = model + (product['predictedProfits'] * decisionVariables[count][product['productName']])
        count = count + 1
    return model

#objective function
model += objectiveFunction(products)
print(model)

In [ ]:
# Define the constraints

 # function to calculate variety of products
def get_variety(products):
    # Create an empty set to store the product names
    product_set = []
    
    # Iterate over the products
    for product in products:
        # Get the product name and add it to the set
        product_name = product['productName']
        product_set.append(product_name)
        
    # Return the length of the set
    return len(product_set)

print(get_variety(products))
variety=get_variety(products)

#see why it results in Infeasible
# model += variety <= maxProductVariety
# model += variety >= minProductVariety


#restriction to total amount of products

def maxMinTotalProductsRestriction(products):
    restrictFunction = 0
    count = 0
    for product in products:
        restrictFunction = restrictFunction + decisionVariables[count][product['productName']]
        count = count + 1
    return restrictFunction
    
model += maxMinTotalProductsRestriction(products) <= maxTotalAmount #restricao de quantidade maximo total de produtos
model += maxMinTotalProductsRestriction(products) >= minTotalAmount #restricao de valor minimo total de produtos

# variety of products

#restriction to maximum quantity of products
def maxMinQuantityProductsRestriction(products,model):
    count = 0
    for product in products:
        model += decisionVariables[count][product['productName']] <= maxAmountProduct
        model += decisionVariables[count][product['productName']] >= minAmountProduct
        count = count + 1
    
maxMinQuantityProductsRestriction(products, model)

print(maxAmountProduct)
print(minAmountProduct)


def maxDimensionProductsRestriction(products):
    count = 0
    restrictFunction = 0
    for product in products:                            
        restrictFunction = restrictFunction + (product['dimension'] * decisionVariables[count][product['productName']])
        count = count +1
    return restrictFunction

print(maxDimensionProductsRestriction(products))
#restricao de dimensões de produtos, menor que dimensao todal da loja
model += maxDimensionProductsRestriction(products) <= storeDimension 

In [ ]:
print(model)

In [ ]:

# Solve the problem and get status
status = model.solve()

print(pulp.LpStatus[status])

In [ ]:
#Optimal solution
def printValuesReturned():
    for v in model.variables():
        try:
            print(v.name,"=", v.value())
        except:
            print("error couldnt find value")
            
printValuesReturned()
print(len(products))
print(products)
print(products[0]['dimension'])

----------- Genetic Algorithm -------------

    Restritions:
    

Restrictions functions return 1 respected, 0 if not respected


In [8]:

#Total Amount Restriction
def validateTotalAmount(individuo):
    somaTotalAmountIndividuo = sum(individuo)
    if (somaTotalAmountIndividuo >= minTotalAmount) and (somaTotalAmountIndividuo <= maxTotalAmount):
        return 1
    else:
        return 0

In [9]:
#Total Quantity Restriction
def validateTotalQuantity(individuo):
    contador = 0
    individuoLen = len(individuo)
    for indAux in individuo:
        if (indAux >= minAmountProduct) and (indAux <= maxAmountProduct):
            contador = contador + 1
    if(contador == individuoLen):
        return 1
    else:
        return 0

In [10]:
#Dimension Restriction
def validateDimension(individuo, products):
    totalDimension = 0
    contador = 0
    for indAux in individuo:
        totalDimension = indAux * products[contador]['dimension']
        contador = contador + 1
    if(totalDimension > storeDimension):
        return 0
    else:
        return 1

Creation of First Population

In [20]:
#Create first population 


#parameters entry:
    #list population []
    #dictionary products [{'productName': 'p1', 'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5},...]
    #populationSize ex:2
#return: list of products with theirs quantities [[3, 3, 3, 1], [3, 1, 3, 3]]
def createFirstPopulation(population, products, populationSize):

    while len(population) < populationSize:
        solution = []
        totalQuantity = 0
        totalDimension = 0
        
        #generates the chromossomes each representing one product
        for j in range(len(products)):
            
            quantity = random.randint(0, round((maxAmountProduct/3)))
            solution.append(quantity)
            
        #Total Amount Restriction
        totalAmountRestriction = validateTotalAmount(solution)
        #Total Quantity Restriction
        totalQuantityRestriction = validateTotalQuantity(solution)
        #Dimension Restriction
        dimensionRestriction = validateDimension(solution, products)
        
        if (totalAmountRestriction == 1) and (totalQuantityRestriction == 1) and (dimensionRestriction == 1):
            population.append(solution)
        
    
    return population


In [21]:
print(createFirstPopulation([],products,2))

[[3, 3, 1, 3], [3, 2, 2, 3]]


Evaluation Function


In [22]:
#get profict of product with quantity 
def getPredictedProfit(product, chosenQuantity):
    return min(product['predictedProfits'], (product['predictedProfits']/product['predictedProductQuantities']) * chosenQuantity)

In [74]:

#Evaluates individuos by their profict

#parameters entry:
    #list population with values: [[1,2,3,4],[1,7,5,4]]
    #dictionary products [{'productName': 'p1', 'predictedProfits': 100, 'predictedProductQuantities': 10, 'dimension': 5},...]
    #listPopulationWithProfit []
    
# returns dictionary with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
def createFitnessFunction(population, products,listPopulationWithIndiviuoAndProfit):

    #population= [[1,2,3,4],[1,7,5,4]]
    for individuo in population:

        individuoProfit = 0
        contador = 0
        #individuo= [1,2,3,4]
        for productQuantity in individuo:

            product = products[contador]
            individuoProfit = individuoProfit + getPredictedProfit(product, productQuantity)
            listPopulationWithProfitAux = {'individuo': individuo, 'calculatedProfit':individuoProfit}
            contador = contador + 1

            
        listPopulationWithIndiviuoAndProfit.append(listPopulationWithProfitAux)


    return listPopulationWithIndiviuoAndProfit

In [75]:
listPopulationWithProfit=createFirstPopulation([],products,10)
listPopulationWithIndiviuoAndProfit=createFitnessFunction(listPopulationWithProfit, products,[])

print(listPopulationWithIndiviuoAndProfit)

[{'individuo': [3, 2, 2, 3], 'calculatedProfit': 80.0}, {'individuo': [2, 3, 3, 2], 'calculatedProfit': 90.0}, {'individuo': [3, 1, 3, 3], 'calculatedProfit': 80.0}, {'individuo': [3, 3, 2, 3], 'calculatedProfit': 90.0}, {'individuo': [2, 2, 3, 3], 'calculatedProfit': 80.0}, {'individuo': [3, 3, 3, 3], 'calculatedProfit': 100.0}, {'individuo': [3, 3, 3, 3], 'calculatedProfit': 100.0}, {'individuo': [3, 2, 2, 3], 'calculatedProfit': 80.0}, {'individuo': [3, 3, 2, 3], 'calculatedProfit': 90.0}, {'individuo': [2, 3, 3, 3], 'calculatedProfit': 90.0}]


Crossover

In [114]:
#Evaluates individuos by their profict

#parameters entry:
    #listPopulationWithIndiviuoAndProfit with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
    
# returns:
    # listPopulationWithProfitOrdered same as listPopulationWithIndiviuoAndProfit but Ordered  
    # populationOrderedByProfit population ordered [[3, 2, 2, 3],..]
def getListsOrderByProfit(listPopulationWithIndiviuoAndProfit):
    listPopulationWithProfitOrdered=sorted(listPopulationWithIndiviuoAndProfit,key=lambda x: x['calculatedProfit'], reverse=True)
    populationOrderedByProfit=[d['individuo'] for d in listPopulationWithProfitOrdered]
    # print(listPopulationWithProfitOrdered)
    # print(populationOrderedByProfit)
    return listPopulationWithProfitOrdered,populationOrderedByProfit

In [115]:
listPopulationWithProfitOrdered, populationOrderedByProfit=getListsOrderByProfit(listPopulationWithIndiviuoAndProfit)
print(listPopulationWithProfitOrdered)
print(populationOrderedByProfit)

[{'individuo': [3, 3, 3, 3], 'calculatedProfit': 100.0}, {'individuo': [3, 3, 3, 3], 'calculatedProfit': 100.0}, {'individuo': [2, 3, 3, 2], 'calculatedProfit': 90.0}, {'individuo': [3, 3, 2, 3], 'calculatedProfit': 90.0}, {'individuo': [3, 3, 2, 3], 'calculatedProfit': 90.0}, {'individuo': [2, 3, 3, 3], 'calculatedProfit': 90.0}, {'individuo': [3, 2, 2, 3], 'calculatedProfit': 80.0}, {'individuo': [3, 1, 3, 3], 'calculatedProfit': 80.0}, {'individuo': [2, 2, 3, 3], 'calculatedProfit': 80.0}, {'individuo': [3, 2, 2, 3], 'calculatedProfit': 80.0}]
[[3, 3, 3, 3], [3, 3, 3, 3], [2, 3, 3, 2], [3, 3, 2, 3], [3, 3, 2, 3], [2, 3, 3, 3], [3, 2, 2, 3], [3, 1, 3, 3], [2, 2, 3, 3], [3, 2, 2, 3]]


In [122]:

#parameters entry:
    #list initialPopulation with values: [[1,2,3,4],[1,7,5,4]] already order by profi
    #list listPopulationGoingToCrossover: []
    #crossoverTax: tax to calculate numer of elements to do crossover ex: 0.10-> 90% do crossover
    #listPopulationWithIndiviuoAndProfit: dictionary with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
    
# returns finalPopulation- list that does not  : [[1,2,3,4],[1,7,5,4]] that will then go to mutation
def getCrossoverListAccordingToCrossoverTax(initialPopulation,crossoverTax):

    numberElementsNoCrossover = round(len(initialPopulation)*crossoverTax)

    #EX:[[1,2,3,4],[1,7,5,4]]
    finalPopulation = [individuo for individuo in initialPopulation[:numberElementsNoCrossover]]
    #EX:[[1,2,3,4],[1,7,5,4]
    listCrossOver = [individuo for individuo in initialPopulation[numberElementsNoCrossover+1:]]

    return finalPopulation,listCrossOver

In [121]:
listPopulationWithProfitOrdered, populationOrderedByProfit=getListsOrderByProfit(listPopulationWithIndiviuoAndProfit)
finalPopulation,listCrossOver=getCrossoverListAccordingToCrossoverTax(populationOrderedByProfit,0.10)
print(finalPopulation)
print(listCrossOver)




[[3, 3, 3, 3]]
[[2, 3, 3, 2], [3, 3, 2, 3], [3, 3, 2, 3], [2, 3, 3, 3], [3, 2, 2, 3], [3, 1, 3, 3], [2, 2, 3, 3], [3, 2, 2, 3]]


In [126]:
#parameters entry:
    #listToCrossOver: [[1,2,3,4],[1,7,5,4]] already order by profi

# returns listpopulationAfterCrossover [[1,2,3,4],[1,7,5,4]] already changed after crossover
def crossoverFunction(listToCrossOver):

    listpopulationAfterCrossover=[]

    while len(listToCrossOver)>0:
    
        if len(listToCrossOver) == 1:
            #listToCrossOver[0]= last individuo remaining [1,2,3,4]
            listpopulationAfterCrossover.append(listToCrossOver[0])
            listToCrossOver.remove(listToCrossOver[0])
            
        else:
            
            #generates random numbers to discover wich individuos ae going to crossover
            indexFirstPai = random.randint(0, len(listToCrossOver)-1)
            indexSecondPai = random.randint(0, len(listToCrossOver)-1)

            #garantees that is not selected the same individu
            while indexFirstPai == indexSecondPai:
                indexSecondPai = random.randint(0, len(listToCrossOver)-1)
            
            #individuo to crossover
            firstPai = listToCrossOver[indexFirstPai]
            secondPai = listToCrossOver[indexSecondPai]
            
            #individuo saved to after remove from list
            firstPaiToDelete = listToCrossOver[indexFirstPai]
            secondPaiToDelete = listToCrossOver[indexSecondPai]
            
            crossoverPoint = random.randint(0, len(listToCrossOver[indexFirstPai])-1)
            
            for i in range(crossoverPoint):
                aux = firstPai[i]
                firstPai[i] = secondPai[i]
                secondPai[i] = aux

             #validate that first chield is good
            totalAmountRestriction1 = validateTotalAmount(firstPai)
            totalQuantityRestriction1 = validateTotalQuantity(firstPai)
            dimensionRestriction1 = validateDimension(firstPai, products)
            #validate that second chield is good
            totalAmountRestriction2 = validateTotalAmount(secondPai)
            totalQuantityRestriction2 = validateTotalQuantity(secondPai)
            dimensionRestriction2 = validateDimension(secondPai, products)
                  
            if (totalAmountRestriction1 == 1) and (totalQuantityRestriction1 == 1) and (dimensionRestriction1 == 1) and (totalAmountRestriction2 == 1) and (totalQuantityRestriction2 == 1) and (dimensionRestriction2 == 1):
                
                listpopulationAfterCrossover.append(firstPai)
                listpopulationAfterCrossover.append(secondPai)
                listToCrossOver.remove(firstPaiToDelete)
                listToCrossOver.remove(secondPaiToDelete)
            

    return listpopulationAfterCrossover
        


In [127]:
listPopulationWithProfitOrdered, populationOrderedByProfit=getListsOrderByProfit(listPopulationWithIndiviuoAndProfit)
finalPopulation,listCrossOver=getCrossoverListAccordingToCrossoverTax(populationOrderedByProfit,0.10)

print(crossoverFunction(listCrossOver))

8
6
4
4
4
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


KeyboardInterrupt: 

In [ ]:
#parameters entry:
    #list population with values: [[1,2,3,4],[1,7,5,4]]
    #crossoverTax: tax to calculate numer of elements to do crossover ex: 0.10-> 90% do crossover
    #listPopulationWithIndiviuoAndProfit: dictionary with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
    
# returns finalPopulation : [[1,2,3,4],[1,7,5,4]] that will then go to mutation and population ordered [[1,2,3,4],[1,7,5,4]]
def crossover(listPopulationWithIndiviuoAndProfit,crossoverTax):

    #function that returns dic ordered [{'individuo': [3, 3, 3, 3], 'calculatedProfit': 100.0}, and 
    listPopulationWithProfitOrdered,populationOrderedByProfit=getListsOrderByProfit(listPopulationWithIndiviuoAndProfit)

    #list with population (size according to crossoverTax) after crossover
    listCrossover=[]
    #function that returns final list of population that doesn't go to crossover[[1,2,3,4],[1,7,5,4]] and list that goes to crossover[[1,2,3,4],[1,7,5,4]]
    finalPopulation,listToCrossOver=getCrossoverListAccordingToCrossoverTax(populationOrderedByProfit,crossoverTax)

    #crossover function that receives list listToCrossOver and returns list after crossover [[1,2,3,4],[1,7,5,4]]
    listpopulationAfterCrossover=crossoverFunction(listToCrossOver)
    
    #population with individuos after crossover being added to the finalpopulation list that is going to go to mutation (size=initialSize)
    finalPopulation.append(listpopulationAfterCrossover)

    return finalPopulation


Mutation

In [ ]:
#parameters entry:
    #list population with values after crossover: [[1,2,3,4],[1,7,5,4]]
    #taxMutation tax to calculate number of elements to do crossover ex: 5-> 5% do mutation
    
# returns listPopulationToMutate : [[1,2,3,4],[1,7,5,4]] after mutation
def getMutationListAccordingToMutationTax(finalPopulation,taxMutation):
    mutationList=finalPopulation[:]
    for x in mutationList:
        #generates random number 
        t=random.randint(0, 100)

        #when random number if > taxMutation then removes values from list, because this is has high probability the list of to mutate is going to be small has expected
        if t >= taxMutation:
            mutationList.remove(x)
    return mutationList

In [ ]:
def mutationFunction(finalPopulation,listPopulationToMutate,taxChangeGene):
    for el in finalPopulation:
        
        initialEl=el
        
        for l in listPopulationToMutate:
            
            if el==l:
                index=random.randint(0, len(listPopulationToMutate[0])-1)
                
                randomSumOrLess = random.randint(0, 1)
                
                if(randomSumOrLess == 1):
                    el[index] = el[index] + round(el[index]*taxChangeGene)
                else:
                    el[index] = el[index] - round(el[index]*taxChangeGene)
                
                        
                totalAmountRestriction = validateTotalAmount(el)
                totalQuantityRestriction = validateTotalQuantity(el)
                dimensionRestriction = validateDimension(el, products)
                        
                if (totalAmountRestriction == 0) or (totalQuantityRestriction == 0) or (dimensionRestriction == 0):
                    el=initialEl
                    
    return finalPopulation

In [ ]:
#parameters entry:
    #list population with values after crossover: [[1,2,3,4],[1,7,5,4]]
    #taxMutation tax to calculate number of elements to do crossover ex: 5-> 5% do mutation
    #changeGene: tax to calculate how the quantity of a product is going to change, ex: 0.1 QP1=2 QP1=2*2*0.1
    
# returns finalPopulation : [[1,2,3,4],[1,7,5,4]] after mutation
def mutation(finalPopulation, taxMutation, taxChangeGene):

    #function that returns final list of population that doesn't go to crossover[[1,2,3,4],[1,7,5,4]] and list that goes to crossover[[1,2,3,4],[1,7,5,4]]
    listPopulationToMutate=getMutationListAccordingToMutationTax(finalPopulation,taxMutation)

    #mutation function that receives list listPopulationToMutate and returns list after mutation [[1,2,3,4],[1,7,5,4]]=Final population to go then to next generation
    listpopulationAfterMutation=mutationFunction(finalPopulation,listPopulationToMutate,taxChangeGene)
    
    return listpopulationAfterMutation

Iterate multiple times

In [ ]:
#parameters entry:
    #list population with values after crossover: [[1,2,3,4],[1,7,5,4]]
    #taxMutation tax to calculate number of elements to do crossover ex: 5-> 5% do mutation
    #changeGene: tax to calculate how the quantity of a product is going to change, ex: 0.1 QP1=2 QP1=2*2*0.1
    
# returns finalPopulation : [[1,2,3,4],[1,7,5,4]] after mutation
def geneticAlgorithm( products, populationSize,crossoverTax,taxMutation,taxChangeGene):

    population=[]
    
    #creates first population with random quantities
    listPopulationWithProfit=createFirstPopulation(population,products,populationSize)

    #creates dic with [{'individuo': [1,2,3,4], 'calculatedProfit': 100},...]
    listPopulationWithIndiviuoAndProfit=createFitnessFunction(listPopulationWithProfit, products,[])
    
    #returns listpopulationAfterCrossover [[1,2,3,4],[1,7,5,4]] already changed after crossover
    listAfterCrossover=crossover(listPopulationWithIndiviuoAndProfit,crossoverTax)

    #returns finalPopulation : [[1,2,3,4],[1,7,5,4]] after mutation
    listAfterMutation=mutation(listAfterCrossover, taxMutation, taxChangeGene)

    return listAfterMutation

    

In [ ]:
def iterate():

    iterations = random.randint(0, 100)
    print(iterations)

    populationSize = random.randint(1,100)
    print(populationSize)

    contador = 0
    while contador < iterations:

        print(populationSize)
        crossoverTax = round(random.uniform(0.01, 0.10),2)
        print(crossoverTax)
        taxMutation = random.randint(1,5)
        print(taxMutation)
        taxChangeGene = round(random.uniform(0.01,0.1),2)
        print(taxChangeGene)

 
        geneticAlgorithm(products, populationSize,crossoverTax,taxMutation,taxChangeGene)
